In [2]:
# Define a vector of package names to be used in the script
package_names_vec <- c(
  "tidyverse",
  "ggtext",
  "showtext",
  "tidymodels",
  "furrr",
  "readxl",
  "tictoc",
  "glmnet",
  "readxl"
)

# Define a function to install and load required packages
package_prep <- function(names_vec) {
  # Iterate through each package name in the provided vector
  for (name in names_vec) {
    # Check if the package is already installed
    if (!require(name, character.only = TRUE)) {
      # Install the package using renv if not already installed
      install.packages(name)
    }
  }
}

# Pipe the vector of package names into the package_prep function
package_names_vec |> package_prep()





if (grepl("kaggle", getwd())) {
  paths <- list(
    data = file.path("/kaggle", "input", "gfi-iran"),
    output = file.path("/kaggle", "working", "output"),
    temp = file.path("/kaggle", "working", "intermediate-data")
  )
} else {
  paths <- list(
    data = file.path("..", "data"),
    output = file.path("..", "output"),
    temp = file.path("..", "intermediate-data")
  )
  for (i in list.files(paths$data, full.names = TRUE)) {
    unzip(i, exdir = paths$data)
  }
  #file.rename(
  #  paths$data |> file.path("IRN_2017_FINDEX_v02_M_EXCEL\\micro_irn_varlabel.xls"),
  #  paths$data |> file.path("micro_irn_varlabel.xls")
  #)
  file.rename(
    paths$data |> file.path("IRN_2017_FINDEX_v02_M_EXCEL\\micro_irn_varname.xls"),
    paths$data |> file.path("micro_irn_varname.xls")
    )
}



# Define a function to load and process a table
# Parameters:
# - name: Name of the table to be assigned in the global environment
# - path: Path to the file to be read
# - argyear: Year to be added as a column in the resulting table
# - reading_func: Function to read the file (e.g., read_csv, read_excel)
load_tbl <- function(name, path, argyear, reading_func) {
  # Read the file, process it, and store it in a variable 'result'
  result <- file.path(path) |> # Generate the full file path
    reading_func() |> # Read the file using the provided reading function
    mutate(
      year = argyear, # Add a 'year' column with the specified year
      ID = row_number() # Add an 'ID' column with sequential row numbers
    ) |>
    select(where(~ !all(is.na(.x)))) # Select only the columns that are not entirely NA

  # Assign the processed table to the specified name in the global environment
  assign(name, result, envir = .GlobalEnv)
}

# Call the 'load_tbl' function to load and process data for the year 2021
load_tbl(
  "gfi_2021_tbl", # Name of the output table
  file.path(paths$data, "micro_irn.csv"), # File path
  2021, # Year to assign
  read_csv # Reading function
)

# Call the 'load_tbl' function to load and process data for the year 2017 (variable labels)
load_tbl(
  "gfi_2017_tbl", # Name of the output table
  file.path(paths$data, "micro_irn_varname.xls"), # File path
  2017, # Year to assign
  read_excel # Reading function
)


Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”
Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”
Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”
Warning message in unzip(i, exdir = paths$data):
“error 1 in extracting from zip file”


[1] TRUE

Rows: 1005 Columns: 84
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): economy, economycode
dbl (78): wpid_random, wgt, female, age, educ, inc_q, emp_in, account, accou...
lgl  (4): urbanicity_f2f, receive_agriculture, remittances, merchantpay_dig

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Load the data and clean

In [55]:
gfi_2021_cleaned_tbl <- gfi_2021_tbl |>
  filter(female == 1) |>
  mutate(
    anydigpayment_s = as.integer(
      account == 1 & (
        account_mob == 1 | # 1
          fin4 == 1 | # 2
          fin8 == 1 | # 3
          (fin31a == 1 | (fin31b == 1 & account_mob == 1)) | # 4
          fin5 == 1 | # 5
          # fin27_1 == 1 | # 6
          (fin14a1 == 1 | fin14a == 1) | # (fin14b == 1 & (fin14c == 1 | fin14c == 3))) | # 7
          # fin29_1 == 1 | # 8
          # (fin43a == 1 | fin43b == 1) | # 9
          (fin34a == 1 | fin34b == 1 | fin34e == 1) | # 10
          ((fin37 == 1 & fin38 != 1) &
            ((fin39a == 1 | fin39e == 1) | fin39b == 1)) | # 11
          ((fin38 == 1) &
            ((fin39a == 1 | fin39e == 1) | fin39b == 1)) | # 12
          (fin14_1 == 1) # | fin8a == 1| fin4a == 1 13
      )
    ),
    pay_utilities_s = case_when(
      fin31a == 1 & !is.na(account_fin) ~ 1L, # from a FI account
      fin31b == 1 & !is.na(account_mob) ~ 1L, # …or from mobile money
      fin31c == 1 ~ 2L, # cash only
      fin30 == 1 ~ 3L, # other (non-cash, non-digital)
      fin30 == 2 ~ 4L, # did not pay
      fin30 %in% c(3, 4) ~ 5L, # DK / refused
      TRUE ~ NA_integer_
    ),
    receive_transfers_s = case_when(
      fin37 == 1 & fin39a == 1 & !is.na(account_fin) ~ 1L, # into FI account
      fin37 == 1 & fin39e == 1 & !is.na(account_fin) ~ 1L, # …or to a card
      fin37 == 1 & fin39b == 1 & !is.na(account_mob) ~ 1L, # …or mobile money
      fin37 == 1 & fin39d == 1 ~ 2L, # cash only
      fin37 == 1 ~ 3L, # other (non-cash, non-digital)
      fin37 == 2 ~ 4L, # did not receive
      fin37 %in% c(3, 4) ~ 5L, # DK / refused
      TRUE ~ NA_integer_
    ),
    receive_pension_s = case_when(
      fin38 == 1 & fin39a == 1 & !is.na(account_fin) ~ 1L, # into FI account
      fin38 == 1 & fin39e == 1 & !is.na(account_fin) ~ 1L, # …or to a card
      fin38 == 1 & fin39b == 1 & !is.na(account_mob) ~ 1L, # …or mobile money
      fin38 == 1 & fin39d == 1 ~ 2L, # cash only
      fin38 == 1 ~ 3L, # other (non-cash, non-digital)
      fin38 == 2 ~ 4L, # did not receive
      fin38 %in% c(3, 4) ~ 5L, # DK / refused
      TRUE ~ NA_integer_
    ),
    receive_wages_s = case_when(
      # 1: received into an account, card, or mobile
      (fin34a == 1 & !is.na(account_fin)) |
        (fin34e == 1 & !is.na(account_fin)) |
        (fin34b == 1 & !is.na(account_mob)) ~ 1,

      # 2: cash only (but only if not already coded 1)
      fin34d == 1 ~ 2,

      # 3: other methods
      fin32 == 1 ~ 3,

      # 4: did not receive payments
      fin32 == 2 ~ 4,

      # 5: don't know / refused
      fin32 %in% c(3, 4) ~ 5,

      # otherwise missing
      TRUE ~ NA_real_
    )
  ) |>
  select(
    year, ID, wgt, age, educ, emp_in, inc_q, fin24, pay_utilities,
    account, account_fin, account_mob, fin22a, borrowed, anydigpayment,
    fin32, fin38, fin30, receive_transfers, receive_pension, receive_pension_s,
    receive_transfers_s, pay_utilities_s, anydigpayment_s, receive_wages_s
  ) |>
  rename(
    id = ID,
    weight = wgt,
    respondent_age = age,
    respondent_education_level = educ,
    household_income_quintile = inc_q,
    in_the_workforce = emp_in,
    main_source_emergency_funds = fin24,
    payments_utility_bills = pay_utilities,
    payments_utility_bills_s = pay_utilities_s,
    has_account = account,
    has_financial_institution_account = account_fin,
    has_mobile_money_account = account_mob,
    borrowed_in_past_year = borrowed,
    borrowed_from_financial_institution = fin22a,
    made_or_received_digital_payment = anydigpayment,
    made_or_received_digital_payment_s = anydigpayment_s,
    payments_wage_payments = receive_wages_s,
    received_government_pension = fin38,
    paid_utility_bill = fin30,
    received_government_transfers = receive_transfers,
    received_government_transfers_s = receive_transfers_s,
    received_government_pension_payment = receive_pension,
    received_government_pension_payment_s = receive_pension_s
  ) |>
  mutate(
    respondent_education_level = respondent_education_level - 1,
    household_income_quintile = household_income_quintile - 1,
    main_source_emergency_funds = main_source_emergency_funds |> case_match(
      2 ~ 0,
      4 ~ 1,
      1 ~ 2,
      3 ~ 3,
      5 ~ 4,
      .default = NA
    ),
    across(
      c(
        payments_utility_bills, payments_utility_bills_s,payments_wage_payments,
        received_government_transfers, received_government_transfers_s,
        received_government_pension_payment, received_government_pension_payment_s
      ),
      ~ case_match(
        .x,
        1 ~ 1,
        2 ~ 2,
        3 ~ 3,
        4 ~ 0,
        .default = NA
      )
    ),
    across(
      c(
        received_government_pension, paid_utility_bill,
        borrowed_from_financial_institution, in_the_workforce
      ),
      ~ case_match(.x, 1 ~ 1, 2 ~ 0)
    )
  )




In [61]:

gfi_2017_cleaned_tbl <- gfi_2017_tbl |>
  filter(female == "Female") |>
  mutate(
    account_mob_s = account_mob,
    across(
      c(
        fin4, fin8, fin5, account, fin31a, fin31b,
        fin14a, fin34a, fin34b, fin34c2, fin37, fin38,
        fin39a, fin39b, fin39c2,fin34a,fin34c2,fin34b,account_mob_s,fin34c1,fin32
      ),
      ~ case_match(.x, "yes" ~ 1, c("no", "0") ~ 0, .default = NA)
    ),
    anydigpayment_s = as.integer(
      account == 1 & (
        # Card and online account usage
        fin4 == 1 | # used debit card in past 12 months :contentReference[oaicite:0]{index=0}
          fin8 == 1 | # used credit card in past 12 months :contentReference[oaicite:1]{index=1}
          fin5 == 1 | # accessed FI account via mobile/internet :contentReference[oaicite:2]{index=2}

          # Utility‐bill payments
          fin31a == 1 | # paid utility by account :contentReference[oaicite:3]{index=3}
          fin31b == 1 | # paid utility by mobile phone :contentReference[oaicite:4]{index=4}

          # Internet bill payments
          fin14a == 1 | # made bill payments online :contentReference[oaicite:5]{index=5}

          # Wage‐payment channels
          fin34a == 1 | # wages into an account :contentReference[oaicite:6]{index=6}
          fin34b == 1 | # wages via mobile phone :contentReference[oaicite:7]{index=7}
          fin34c2 == 1 | # wages to a card :contentReference[oaicite:8]{index=8}

          # Government transfers (non-pension) via digital channels
          (fin37 == 1 & fin38 != 1) &
            (fin39a == 1 | fin39b == 1 | fin39c2 == 1) | # government transfer (not pension) :contentReference[oaicite:9]{index=9}

          # Government pensions via digital channels
          (fin38 == 1) &
            (fin39a == 1 | fin39b == 1 | fin39c2 == 1) # government pension :contentReference[oaicite:10]{index=10}
      )
    ),
    receive_wages_s = case_when(
      # 1: received into an account
      fin34a == 1 & !is.na(account_fin) ~ '1',
      # also 1: received onto a card
      fin34c2 == 1 & !is.na(account_fin) ~ '1',
      # also 1: received via mobile
      fin34b == 1 & !is.na(account_mob_s) ~ '1',
      # 2: cash only (only reached if none of the above matched)
      fin34c1 == 1 ~ '2',
      # 3: other methods only
      fin32 == 1 ~ '3',
      # 4: did not receive payments
      fin32 == 0 ~ '4',
      # 5: don't know / refused
      is.na(fin32) ~ '5',
      # else: missing
      TRUE ~ NA_character_
    ),
  ) |>
  select(
    year, ID, wgt, age, educ, emp_in, inc_q, fin25, pay_utilities,
    account, account_fin, account_mob, fin22a, borrowed, anydigpayment_s,
    receive_wages_s, fin38, fin30, receive_transfers, receive_pension
  ) |>
  rename(
    id = ID,
    weight = wgt,
    respondent_age = age,
    respondent_education_level = educ,
    household_income_quintile = inc_q,
    in_the_workforce = emp_in,
    main_source_emergency_funds = fin25,
    payments_utility_bills = pay_utilities,
    has_account = account,
    has_financial_institution_account = account_fin,
    has_mobile_money_account = account_mob,
    borrowed_in_past_year = borrowed,
    borrowed_from_financial_institution = fin22a,
    payments_wage_payments = receive_wages_s,
    received_government_pension = fin38,
    paid_utility_bill = fin30,
    received_government_transfers = receive_transfers,
    received_government_pension_payment = receive_pension,
    made_or_received_digital_payment_s = anydigpayment_s
  ) |>
  mutate(
    respondent_education_level = respondent_education_level |> case_match(
      c("(rf)", "(dk)") ~ NA,
      "completed primary or less" ~ 0,
      "secondary" ~ 1,
      "completed tertiary or more" ~ 2
    ),
    household_income_quintile = household_income_quintile |> case_match(
      "Poorest 20%" ~ 0,
      "Middle 20%" ~ 1,
      "Fourth 20%" ~ 2,
      "Second 20%" ~ 3,
      "Richest 20%" ~ 4
    ),
    in_the_workforce = in_the_workforce |> case_match(
      "out of workforce" ~ 0,
      "in workforce" ~ 1
    ),
    main_source_emergency_funds = main_source_emergency_funds |> case_match(
      "Main source: Family or friends" ~ 0,
      "Main source: Borrowing from a bank, empoyer, or private lender" ~ 1,
      "Main source: Savings" ~ 2,
      "Main source: Money from working" ~ 3,
      "Main source: Selling assets" ~ 4,
      .default = NA
    ),
    across(
      c(received_government_pension_payment, payments_utility_bills, received_government_transfers,payments_wage_payments),
      ~ case_match(.x, "1" ~ 1, "2" ~ 2, "3" ~ 3, "4" ~ 0, .default = NA)
    ),
    # recieve government pension payments
    # 1 "into an account" 2 "in cash" 3 "using other methods only" 0 "did not receive payments" NA "(dk)/ref",
    # recieve government transfers
    # 1 "into an account" 2 "in cash" 3 "using other methods only" 0 "did not receive payments" NA "(dk)/ref"
    # payments utility bills
    # 1 "into an account" 2 "in cash" 3 "using other methods only" 0 "did not receive payments" NA "(dk)/ref",
    across(
      c(
        paid_utility_bill, has_financial_institution_account, has_mobile_money_account,
        borrowed_in_past_year, borrowed_from_financial_institution
      ),
      ~ case_match(
        .x,
        c("no", "0") ~ 0,
        "yes" ~ 1,
        .default = NA
      )
    ),
    respondent_age = case_when(respondent_age == "99+" ~ "99", TRUE ~ respondent_age),
    across(-c(weight), ~ as.integer(.x))
  )



In [62]:


gfi_tbl <- bind_rows(gfi_2017_cleaned_tbl, gfi_2021_cleaned_tbl)


In [68]:
gfi_tbl |> map(~unique(.x))

$year
[1] 2017 2021

$id
  [1]    2    4    8    9   11   13   14   16   18   19   20   24   26   27   29
 [16]   35   36   37   40   41   42   43   45   46   50   51   55   56   59   60
 [31]   61   63   64   66   69   70   74   75   76   78   79   80   81   83   84
 [46]   86   87   89   90   93   94   95   96   98  100  103  105  107  109  112
 [61]  113  114  115  116  117  118  122  123  125  126  127  129  130  131  133
 [76]  134  135  136  137  138  140  141  144  145  147  148  149  150  153  154
 [91]  155  156  159  160  161  162  163  164  166  168  169  170  171  173  175
[106]  176  177  181  182  184  186  187  189  191  192  193  195  196  197  199
[121]  200  201  202  205  212  216  217  221  223  226  229  231  232  233  234
[136]  238  243  245  249  250  251  256  257  261  262  263  264  265  271  272
[151]  273  279  280  287  289  290  291  293  297  299  301  304  308  309  310
[166]  315  317  318  319  320  322  324  328  329  330  331  335  338  341  342
[181]  343  345  346  347  348  353  354  357  359  360  361  362  363  364  365
[196]  366  371  372  373  374  375  376  377  378  380  382  384  386  388  389
[211]  390  391  392  399  401  402  403  406  407  411  412  413  414  415  418
[226]  420  422  423  424  425  426  427  428  432  437  438  439  441  442  444
[241]  445  447  454  455  456  457  461  462  464  467  469  470  471  472  475
[256]  476  478  479  483  484  485  487  488  489  490  492  493  495  497  498
[271]  499  501  503  505  507  508  513  514  516  517  518  521  522  524  526
[286]  530  532  533  535  540  541  542  544  545  547  548  553  554  555  558
[301]  566  567  571  573  576  577  578  579  580  583  586  587  594  597  598
[316]  599  600  602  603  605  607  609  611  612  613  614  615  617  622  624
[331]  630  633  636  638  639  641  642  643  645  647  648  650  652  654  658
[346]  660  661  662  665  666  667  668  669  670  671  673  676  678  680  681
[361]  682  683  684  685  686  687  689  692  693  694  696  698  700  701  703
[376]  704  705  706  707  709  710  712  714  715  719  725  726  728  729  732
[391]  733  734  737  738  739  740  741  743  744  748  750  751  752  753  756
[406]  757  758  764  766  767  768  770  771  772  775  776  777  778  780  783
[421]  785  787  789  790  791  792  794  795  797  798  799  801  802  803  804
[436]  806  812  814  815  816  818  820  822  828  829  830  833  834  835  839
[451]  845  847  848  849  850  851  856  858  860  863  865  868  869  872  873
[466]  874  875  878  880  881  884  885  886  888  889  890  891  892  893  894
[481]  895  898  899  902  903  905  909  910  911  913  915  916  917  918  921
[496]  923  926  927  928  929  931  932  934  935  936  937  938  940  943  946
[511]  947  949  950  951  952  954  956  959  961  964  965  969  970  972  974
[526]  975  976  982  983  984  985  988  990  991  992  994  996  997 1002    3
[541]    6    7   12   15   17   21   22   25   31   34   47   48   52   65   71
[556]   72   73   77   88   92   97  106  111  124  139  143  146  151  157  165
[571]  172  178  179  183  185  188  190  194  206  209  210  213  218  230  236
[586]  237  240  241  242  253  254  255  267  268  269  275  276  277  278  281
[601]  282  285  292  300  302  303  306  311  314  321  323  327  333  350  352
[616]  356  367  368  369  370  379  383  393  394  395  398  404  405  410  417
[631]  429  431  433  434  435  440  446  448  451  453  458  463  468  473  482
[646]  486  491  494  496  502  504  506  509  511  512  523  525  528  529  538
[661]  539  543  546  549  550  556  559  562  565  569  572  574  575  581  582
[676]  584  585  589  593  595  601  606  608  610  618  619  625  626  627  631
[691]  632  637  640  644  651  653  655  657  663  674  675  679  688  697  699
[706]  711  713  716  721  723  724  730  731  735  745  746  747  754  763  773
[721]  784  793  800  805  807  808  813  817  819  823  825  831  840  843  844
[736] 

# Models

## Paremtric without Bootsrtap

In [ ]:
weighted_sumw <- combined_tbl |>
  mutate(
    Within_economy_household_income_quintile = factor(
      Within_economy_household_income_quintile,
      levels = c("Poorest 20%", "Second 20%", "Middle 20%", "Fourth 20%", "Richest 20%"),
      ordered = TRUE
    ),
    Respondent_education_level = factor(
      Respondent_education_level,
      levels = c("completed primary or less", "secondary", "completed tertiary or more"),
      ordered = TRUE
    )
  )
workflow() |>
  add_model(
    logistic_reg() |> set_engine("glm") |> set_mode("classification")
  ) |>
  add_recipe(
    recipe(
      Respondent_is_in_the_workforce ~
        Respondent_is_female +
        Respondent_age +
        Has_an_account_at_a_financial_institution +
        Has_a_mobile_money_account +
        Within_economy_household_income_quintile +
        year +
        Respondent_education_level,
      data = weighted_sumw,
      weights = weight
    ) |>
      step_dummy(c(
        year, Respondent_is_female, Has_an_account_at_a_financial_institution, Has_a_mobile_money_account
        )) |>
      step_ordinalscore(c(Within_economy_household_income_quintile, Respondent_education_level)) |>
      step_interact(~ .:starts_with("year"))
  ) |>
  fit(data = weighted_sumw) |>
  extract_fit_parsnip() |>
  tidy()


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.630387769,5.791085e-02,-1.088549e+01,1.351747e-27
Respondent_age,-0.007826444,6.264152e-04,-1.249402e+01,8.048189e-36
Within_economy_household_income_quintile,-0.030123903,6.809100e-03,-4.424065e+00,9.686082e-06
Respondent_education_level,0.559857790,1.479642e-02,3.783738e+01,0.000000e+00
year_X2021,-18.935680750,2.744874e+02,-6.898561e-02,9.450011e-01
Respondent_is_female_Female,-1.696339211,1.908269e-02,-8.889412e+01,0.000000e+00
Has_an_account_at_a_financial_institution_Yes,0.749857829,4.148945e-02,1.807346e+01,5.157652e-73
Has_a_mobile_money_account_Yes,0.461204722,2.173457e-02,2.121987e+01,6.259683e-100
Respondent_age_x_year_X2021,0.007826444,3.088155e+00,2.534343e-03,9.979779e-01
